# Setup
First of all, we install all required libraries

In [1]:
!pip install -r requirements.txt --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Inference
Here we show how to use the framework for our pretrained models

In [21]:
import os

data_dir = "data"
dump_dir = os.path.join(data_dir, "wikimedia_dumps")
extracted_dir = os.path.join(data_dir, "extracted_txt")
model_dir = "models"
os.makedirs(dump_dir, exist_ok=True)
os.makedirs(extracted_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
n = 3

In [22]:
from scripts.utils import load_model
from scripts.classify import detect_language

In [23]:
def load_language_models(model_dir):
    """Load language models from disk."""
    language_models = {}

    for model_file in os.listdir(model_dir):
        if model_file.endswith("_model.pkl"):
            language = model_file.split("_model.pkl")[0]
            model_path = os.path.join(model_dir, model_file)
            language_models[language] = load_model(model_path)
            print(f"Model loaded for {language}: {model_path}")

    return language_models

In [24]:
def classify_text(input_text, language_models, n):
    """Classify the language of the input text."""
    probabilities = detect_language(input_text, language_models, n)
    return max(probabilities, key=probabilities.get), probabilities

In [25]:
loaded_models = load_language_models(model_dir)

Model loaded for de: models\de_model.pkl
Model loaded for en: models\en_model.pkl
Model loaded for fr: models\fr_model.pkl
Model loaded for it: models\it_model.pkl


Here is the inference in action

In [28]:
input_text = """What? I can tell you something"""

In [29]:
detected_language, probabilities = classify_text(input_text, loaded_models, n)
print(f"Detected language: {detected_language}\nProbabilities: {probabilities}")

Detected language: en
Probabilities: {'de': np.float64(0.2356590678050734), 'en': np.float64(0.29182772922628053), 'fr': np.float64(0.21953500163136253), 'it': np.float64(0.25297820133728366)}


In [30]:
input_text = """**Die Schönheit der Natur und ihre Bedeutung für den Menschen**

Die Natur ist ein unerschöpflicher Quell der Inspiration und des Staunens. Von den majestätischen Bergen, die in den Himmel ragen, bis zu den stillen Wäldern, deren Baumkronen ein grünes Dach bilden, hat die Natur stets eine beruhigende und gleichzeitig belebende Wirkung auf den Menschen. Sie bietet nicht nur einen Rückzugsort aus dem hektischen Alltag, sondern erinnert uns auch an die Einfachheit und den Reichtum des Lebens.

In einer Welt, die zunehmend von Technologie und urbanem Lebensstil geprägt ist, scheint der Kontakt zur Natur manchmal verloren zu gehen. Doch gerade dieser Kontakt ist essenziell für das Wohlbefinden von Körper und Geist. Zahlreiche Studien belegen, dass Aufenthalte in der Natur Stress reduzieren, die Kreativität fördern und die allgemeine Lebenszufriedenheit steigern können. Ein einfacher Spaziergang im Park oder eine Wanderung in den Bergen reichen oft aus, um neue Energie zu tanken und den Kopf freizubekommen.

Die Natur ist jedoch nicht nur ein Ort der Erholung, sondern auch eine unverzichtbare Lebensgrundlage. Sie liefert uns Nahrung, Wasser und Luft – die Grundelemente des Lebens. Gleichzeitig ist sie ein komplexes Ökosystem, in dem jedes Lebewesen, sei es noch so klein, eine wichtige Rolle spielt. Der Verlust von Artenvielfalt und die Zerstörung von Lebensräumen haben weitreichende Konsequenzen, die nicht nur Tiere und Pflanzen, sondern auch den Menschen betreffen. Daher ist es von großer Bedeutung, die Natur zu schützen und nachhaltige Lebensweisen zu fördern.

Ein weiteres faszinierendes Element der Natur ist ihre Fähigkeit zur Regeneration. Auch wenn der Mensch sie oft stark beansprucht und sogar zerstört, zeigt sie immer wieder ihre unglaubliche Widerstandsfähigkeit. Ein Wald, der nach einem Brand wieder ergrünt, oder ein Fluss, der sich nach Jahren der Verschmutzung erholt, sind eindrucksvolle Beispiele dafür. Diese Regenerationsfähigkeit sollte uns jedoch nicht dazu verleiten, sorglos mit der Natur umzugehen. Vielmehr sollte sie uns daran erinnern, dass wir eine Verantwortung tragen, sie für kommende Generationen zu bewahren.

Auch kulturell und spirituell spielt die Natur eine zentrale Rolle. Viele Völker und Religionen verehren die Natur als göttlich oder sehen in ihr eine Verbindung zum Höheren. Alte Mythen und Legenden ranken sich um Berge, Flüsse und Wälder, und auch in der modernen Kunst und Literatur ist die Natur ein häufiges Motiv. Diese Verbindung zeigt, wie tief verwurzelt die Beziehung zwischen Mensch und Natur ist.

Abschließend bleibt zu sagen, dass die Natur nicht nur ein Ort des Rückzugs oder ein funktionaler Bestandteil unseres Lebens ist, sondern auch eine Quelle der Inspiration, der Erkenntnis und des Friedens. Indem wir uns die Zeit nehmen, sie zu erleben und zu schützen, gewinnen wir nicht nur ein tieferes Verständnis für unsere Umwelt, sondern auch für uns selbst."""

More data gives our model more certainty

In [31]:
detected_language, p = classify_text(input_text, loaded_models, n)
print(f"Detected language: {detected_language}, probabilities: {p}")

Detected language: de, probabilities: {'de': np.float64(0.9999999056499485), 'en': np.float64(3.395311146388299e-08), 'fr': np.float64(3.3915970895378446e-08), 'it': np.float64(2.648096902140539e-08)}


# Adding new languages and training data

Next, we want to download some training data, in order to learn language-representations.

The strategy is to build n-gram models of each language, and use that model as a dense representation of that language.
We then build an n-gram model of the document to identify, and compare which of the trained models is the closest.

Note: The wikimedia data quality unfortunately is a little messed up by the wikimedia foundation, so the training data could be much more improved. Despite this, the model should pick up the language we are looking for.

In [32]:
import requests
from tqdm import tqdm

def download_file(url, output_path, language_code):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total_size = int(response.headers.get('content-length', 0))
    with tqdm(total=total_size, unit='B', unit_scale=True, desc=f"Downloading {language_code}", leave=True) as pbar:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1 MB chunks
                f.write(chunk)
                pbar.update(len(chunk))

In [33]:
def download_wikimedia_dump(language_code, dump_dir):
    """Download Wikimedia dump for a specific language."""
    filename = f"{language_code}wiki-latest-abstract.xml.gz"
    dump_url = f"https://dumps.wikimedia.org/{language_code}wiki/latest/{filename}"
    dump_file = os.path.join(dump_dir, filename)
    if not os.path.exists(dump_file):
        download_file(dump_url, dump_file, language_code)
    else:
        print(f"Dump already exists for {language_code}.")
    return dump_file

In [34]:
import os
import gzip
import xml.etree.ElementTree as ET
from tqdm import tqdm

def extract_text_from_dump(dump_file, output_dir, language_code, max_docs=5000):
    if not os.path.exists(output_dir): os.makedirs(output_dir)

    output_file = os.path.join(output_dir, f"{language_code}.txt")

    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping processing.")
        return

    # Estimate the total number of <doc> tags if no max_docs is specified
    total_docs = 0
    with gzip.open(dump_file, 'rt', encoding='utf-8') as f:
        for line in f:
            if "<doc>" in line: total_docs += 1
            if total_docs >= max_docs: break

    with gzip.open(dump_file, 'rt', encoding='utf-8') as f:
        context = ET.iterparse(f, events=('start', 'end'))
        _, root = next(context)  # Get the root element

        cnt = 0
        with tqdm(total=total_docs, desc="Extracting articles", unit="doc", leave=True) as pbar, \
            open(output_file, "w", encoding="utf-8") as out_file:
            for event, elem in context:
                if cnt >= max_docs: break
                if event == 'end' and elem.tag == 'doc':
                    abstract = elem.find('abstract').text

                    if abstract: out_file.write(abstract.strip() + "\n")  # Write the abstract to the output file

                    pbar.update(1)
                    cnt += 1

                    # Clear the processed element to save memory
                    root.clear()

We can now download and prepare our datasets. Only do this if you want to train a new model! Otherwise the models are already included in the repo

In [35]:
language_codes = ["en", "de", "fr", "it"] # extend as wished
for language_code in language_codes:
    dump_file = download_wikimedia_dump(language_code, dump_dir)
    extracted_text_dir = extract_text_from_dump(dump_file, extracted_dir, language_code)

Dump already exists for en.
Output file data\extracted_txt\en.txt already exists. Skipping processing.
Dump already exists for de.
Output file data\extracted_txt\de.txt already exists. Skipping processing.
Dump already exists for fr.
Output file data\extracted_txt\fr.txt already exists. Skipping processing.
Dump already exists for it.
Output file data\extracted_txt\it.txt already exists. Skipping processing.


# Model training

Now, we import all the required functionality for model training and comparison. For implementation details, please check the respective files.

In [36]:
from scripts.data_loader import load_wikimedia_texts
from scripts.preprocessing import preprocess_text
from scripts.train import train_language_model
from scripts.utils import save_model

# Load and process Wikimedia
Our language identifier works by building an n-gram model of the document to classify, and compare it to previously trained n-gram models of various languages.
These languages are pulled from wikipedia


In [37]:
def load_and_preprocess_all_languages(extracted_dir):
    language_texts = {}

    for file_name in os.listdir(extracted_dir):
        if file_name.endswith(".txt"):  # Process only .txt files
            lang_code = os.path.splitext(file_name)[0]  # Extract language code from filename
            file_path = os.path.join(extracted_dir, file_name)

            print(f"Processing language: {lang_code}")

            # Load all lines from the file
            with open(file_path, "r", encoding="utf-8") as f:
                texts = f.readlines()  # Each line corresponds to one abstract

            # Preprocess the texts
            preprocessed_texts = [preprocess_text(text.strip()) for text in texts]
            language_texts[lang_code] = preprocessed_texts

    return language_texts


In [38]:
def save_language_models(language_models, model_dir):
    """Save language models to disk."""
    os.makedirs(model_dir, exist_ok=True)

    for language, model in language_models.items():
        model_path = os.path.join(model_dir, f"{language}_model.pkl")
        save_model(model, model_path)
        print(f"Model saved for {language}: {model_path}")

In [39]:
def train_language_models(language_texts, n):
    """Train n-gram models for all languages."""
    language_models = {}

    for language, texts in language_texts.items():
        print(f"Training model for {language}")
        language_models[language] = train_language_model(texts, n)

    return language_models

In [40]:
language_texts = load_and_preprocess_all_languages(extracted_dir)
language_models = train_language_models(language_texts, n)
save_language_models(language_models, model_dir)

Processing language: de
Processing language: en
Processing language: fr
Processing language: it
Training model for de
Training model for en
Training model for fr
Training model for it
Model saved for de: models\de_model.pkl
Model saved for en: models\en_model.pkl
Model saved for fr: models\fr_model.pkl
Model saved for it: models\it_model.pkl


# Inference

Finally, we can load the new models and run language identification on any text

In [41]:
loaded_models = load_language_models(model_dir)

Model loaded for de: models\de_model.pkl
Model loaded for en: models\en_model.pkl
Model loaded for fr: models\fr_model.pkl
Model loaded for it: models\it_model.pkl


In [42]:
def print_classification(input_text):
    detected_language, probabilities = classify_text(input_text, loaded_models, n)
    print(f"Detected language: {detected_language}\nProbabilities: {probabilities}")

In [43]:
print_classification("""La lecture est""") # Please include at least 3 words

Detected language: fr
Probabilities: {'de': np.float64(0.23455395834068382), 'en': np.float64(0.24990902768443918), 'fr': np.float64(0.2658008928316076), 'it': np.float64(0.24973612114326935)}


Congrats, you completed the entire notebook!